In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Загрузка датасета**

In [ ]:
data = pd.read_csv('/kaggle/input/student-study-performance/study_performance.csv')

**1. Предварительный анализ данных**

In [ ]:
print(data.head())
print(data.describe())

Таблица отображает два разных вида статистической информации о датасете:

data.head() показывает первые пять строк датасета, как пол (gender), этническая принадлежность (race/ethnicity), уровень образования родителей (parental level of education), завтрак (lunch), прохождение подготовительных курсов (test preparation course) и баллы по математике (math score), чтению (reading score) и письму (writing score).

data.describe() предоставляет описательную статистику для всех числовых столбцов в датасете. Это включает:
-Количество значений (count)
-Среднее значение (mean)
-Стандартное отклонение (std), показывающее разброс данных
-Минимальное значение (min)
-25-й перцентиль (25%), который является первым квартилем
-Медианное значение (50%), также известное как второй квартиль
-75-й перцентиль (75%), который является третьим квартилем
-Максимальное значение (max)

**2. Исследование данных**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Визуализация только числовых переменных
data_numeric = data.select_dtypes(include=[np.number])
data_numeric.hist(bins=15, figsize=(15, 10))
plt.show()

# Корреляционная матрица только для числовых переменных
plt.figure(figsize=(10, 8))
sns.heatmap(data_numeric.corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.show()

**3. Очистка данных**

In [ ]:
# Пропущенные значения отсутствуют на предоставленном снимке экрана,
# но если они есть, можно их заполнить таким образом:
# data['column_name'] = data['column_name'].fillna(data['column_name'].mean()) # для численных столбцов
# data['column_name'] = data['column_name'].fillna(data['column_name'].mode()[0]) # для категориальных столбцов

# Кодируем категориальные переменные
# One-Hot Encoding для переменных с малым числом категорий
data = pd.get_dummies(data, columns=['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch', 'test_preparation_course'])

# Масштабирование признаков, для этого можно использовать StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Масштабируем только числовые столбцы
scaled_columns = scaler.fit_transform(data[['math_score', 'reading_score', 'writing_score']])
data[['math_score', 'reading_score', 'writing_score']] = scaled_columns

# Проверяем результаты
print(data.head())


Эти данные представляют обработанный датасет, где:
-math_score, reading_score, writing_score - это стандартизированные баллы по математике, чтению и письму. Стандартизация преобразует распределение оценок так, чтобы среднее значение было 0, а стандартное отклонение — 1.
-gender_female, gender_male - это флаги, полученные после one-hot кодирования пола, где True означает, что учащийся соответствует категории, а False - не соответствует.
-столбцы (race_ethnicity_group A, race_ethnicity_group B и т.д.) также являются результатом one-hot кодирования и показывают принадлежность учащегося к определенной этнической группе.
-столбцы, связанные с parental_level_of_education_..., указывают на уровень образования родителей.
-lunch_free/reduced, lunch_standard - указывают на тип питания учащегося, где True означает, что данный тип питания учащегося соответствует названию столбца.
-test_preparation_course_completed, test_preparation_course_none - показывают, завершил ли учащийся подготовительные курсы.
Значение True или False для каждой из этих dummy-переменных показывает, принадлежит ли наблюдение к соответствующей категории.

**4. Формирование гипотез и тестирование**

1. T-тест для сравнения математических оценок по курсу подготовки - это помогает определить, влияет ли участие в подготовительных курсах на успеваемость по математике.
2. Исследование зависимостей между категориальными и числовыми переменными - позволяет выявить, как различные категориальные признаки (например, пол или этническая принадлежность) влияют на числовые результаты (оценки).
3. Анализ распределений каждого признака в разрезе различных групп - изучает, как признаки распределяются среди разных групп, что может помочь выявить паттерны или необходимость корректировки данных.
4. Создание сводных таблиц для сравнения средних значений между различными группами - обеспечивает наглядное сравнение, которое может быть полезно для подтверждения или опровержения гипотез.

T-тест для сравнения математических оценок по курсу подготовки

In [ ]:
from scipy.stats import ttest_ind

# Сегментируем оценки по группам
group_completed = data[data['test_preparation_course_completed'] == True]['math_score']
group_none = data[data['test_preparation_course_none'] == True]['math_score']

# Применяем t-тест
t_stat, p_val = ttest_ind(group_completed, group_none)

# Выводим результаты
print(f"T-Statistic: {t_stat}, P-Value: {p_val}")

# Интерпретация результатов
alpha = 0.05  # Уровень значимости
if p_val < alpha:
    print("Отклоняем нулевую гипотезу, существует значимая разница.")
else:
    print("Не отклоняем нулевую гипотезу, значимой разницы нет.")


P-значение в результате говорит о вероятности получить данные, как минимум такие же экстремальные, как те, что были получены во время теста, если нулевая гипотеза верна. Очень маленькое P-значение, такое как 1.53e-08 (что эквивалентно 0.00000001539), гораздо меньше стандартного порога α = 0.05, и поэтому можно сделать вывод о том, что есть статистически значимая разница между группами.

Значение T-статистики 5.704 указывает на то, насколько велико отклонение средних значений между группами в единицах стандартного отклонения. Большое значение T-статистики в сочетании с маленьким P-значением подтверждает, что различия между группами значимы с точки зрения статистики.

Исследование зависимостей между категориальными и числовыми переменными

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.boxplot(x='test_preparation_course_completed', y='math_score', data=data)
plt.title('Влияние подготовительных курсов на оценки по математике')
plt.show()


Анализ распределений каждого признака в разрезе различных групп

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
import seaborn as sns
import matplotlib.pyplot as plt

# Гистограмма для мужчин
sns.histplot(data[data['gender_male'] == True]['math_score'], color="blue", label='Male', kde=True, stat="density", linewidth=0)

# Гистограмма для женщин
sns.histplot(data[data['gender_female'] == True]['math_score'], color="red", label='Female', kde=True, stat="density", linewidth=0)

plt.legend(title='Gender')
plt.title('Distribution of Math Scores by Gender')
plt.xlabel('Math Score')
plt.ylabel('Density')
plt.show()

Этот код создает две наложенные гистограммы для мужчин и женщин, что позволяет визуально сравнить распределение оценок по математике между двумя группами. Аргумент stat="density" гарантирует, что площадь под гистограммой нормализуется, что делает сравнение более честным, особенно если размеры групп различаются.

Создание сводных таблиц для сравнения средних значений между различными группами

In [ ]:
import pandas as pd
mean_scores = pd.pivot_table(data, values=['math_score', 'reading_score', 'writing_score'], index=['gender_male'], aggfunc='mean')
print(mean_scores)

Таблица, представляет средние значения оценок по математике, чтению и письму для разных групп по полу (мужчины и женщины), где данные были стандартизированы (среднее = 0, стандартное отклонение = 1). Столбцы math_score, reading_score, и writing_score показывают средние значения для каждой группы. False обозначает женщин, True — мужчин.

**5. Создание окончательной визуализации для истории**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Предполагая, что вы преобразовали 'parental_level_of_education' в столбцы формата 'parental_level_of_education_<category>'
education_levels = [col for col in data.columns if col.startswith('parental_level_of_education_')]

plt.figure(figsize=(15, 10))
for i, level in enumerate(education_levels, 1):
    plt.subplot(2, 3, i)  # Предполагая, что у нас не больше 6 категорий образования
    sns.boxplot(x=data[level], y=data['math_score'])
    plt.title(f'Оценки по математике: {level.split("_")[-1]}')
    plt.xlabel('Присутствие уровня образования')
    plt.ylabel('Оценки по математике')
    if i == len(education_levels):
        plt.xticks([0, 1], ['Нет', 'Да'])
    else:
        plt.xticks([])  # Скрываем подписи x-оси для наглядности

plt.tight_layout()
plt.show()


Этот код создает отдельные диаграммы для каждой категории образования родителей и показывает распределение оценок по математике для студентов, чьи родители соответствуют этой категории образования. 

Создадим диаграмму рассеяния (scatter plot), которая может помочь визуализировать взаимосвязь между различными переменными в вашем датасете. Например, можно исследовать зависимость между оценками по математике и оценками по чтению, разделяя данные по наличию или отсутствию подготовительных курсов.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Диаграмма рассеяния, показывающая взаимосвязь между оценками по математике и чтению
# разделяем по наличию подготовительных курсов
plt.figure(figsize=(10, 6))
sns.scatterplot(x='math_score', y='reading_score', hue='test_preparation_course_completed', data=data, style='test_preparation_course_completed', palette='bright')
plt.title('Взаимосвязь оценок по математике и чтению в зависимости от подготовительных курсов')
plt.xlabel('Оценка по математике')
plt.ylabel('Оценка по чтению')
plt.legend(title='Завершил подготовительные курсы')
plt.grid(True)
plt.show()

Этот график поможет визуально оценить, есть ли различия в оценках по математике и чтению между студентами, которые завершили подготовительные курсы, и теми, кто их не проходил.

**ВЫВОД:**
На основе выполненных задач, таких как анализ данных, тестирование гипотез, визуализация результатов, и сводные анализы, можно считать, что лабораторная работа по созданию "истории о данных" выполнена успешно. 
Был проведен комплексный анализ, используя различные методы статистической обработки и визуализации данных, что позволило выявить значимые закономерности и подтвердить сформулированные гипотезы.